In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample, evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import pickle
import warnings
import math
import torch

warnings.filterwarnings("ignore")

In [31]:
df = pd.read_csv("Data/sample_test.csv", delimiter='\t', encoding="utf-8").dropna()

sentences1 = []
sentences2 = []
indexes = []

def map_cos_to_zero_one(x):
    mapped_value = (x + 1) / 2
    return mapped_value

for index, row in df.iterrows():
    indexes.append(row["id"])
    sentences1.append(row["setence1"])
    sentences2.append(row["sentence2"])

with open('checkpoints/model_1C.pkl', 'rb') as f:
    model = pickle.load(f)

embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)
cosine_scores = util.cos_sim(embeddings1, embeddings2)
predictions = []
for i in range(len(cosine_scores)):
    predictions.append(cosine_scores[i][i])
transformed_predictions = [5 * map_cos_to_zero_one(value) for value in predictions]

print("id\tscore\tsentence1\tsentence2\n")
with open('Data/sample_demo.csv', 'w', encoding='utf-8') as f:
    f.write("id\tscore\tsentence1\tsentence2\n")
    for i in range(len(predictions)):
        print(f"{indexes[i]}\t{transformed_predictions[i]}\t{sentences1[i]}\t{sentences2[i]}\n")
        f.write(f"{indexes[i]}\t{transformed_predictions[i]}\t{sentences1[i]}\t{sentences2[i]}\n")

id	score	sentence1	sentence2

1	4.979576587677002	A man with a hard hat is dancing.	A man wearing a hard hat is dancing.

2	4.829216480255127	A young child is riding a horse.	A child is riding a horse.

3	4.952564716339111	A man is feeding a mouse to a snake.	The man is feeding a mouse to the snake.

4	4.33746862411499	A woman is playing the guitar.	A man is playing guitar.

5	4.576325416564941	A woman is playing the flute.	A man is playing a flute.

6	4.415567398071289	A woman is cutting an onion.	A man is cutting onions.

